Esse notebook tem como objetivo explicar os conceitos e a aplicação do WOE e IV no processo de elaboração de um credit score.

## 1. Importando as bibliotecas

In [1]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import Image
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
diretorio=os.getcwd()
diretorio

'/Users/igoracmorais/igor/4labs/consulting/score_credito/explicando_scorecard'

## 2. Dados
Os dados usados aqui são os salvos no notebook 1 após o preenchimento dos nulos.

In [2]:
df = pd.read_csv('german_credit_data2.csv',index_col=0)
print(df.shape)
df.head(2)

(1000, 10)


,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,default
0,67,male,2,own,little,little,1169,6,radio/TV,yes
1,22,female,2,own,little,moderate,5951,48,radio/TV,no


## Parte 1. Fazendo o WOE "na mão"

O WOE mede o poder de previsao de uma variavel independente considerando que a variavel objetivo é dividida em classes. Nesse caso, as variveis continuas, como renda e idade, por exemplo, sao transformadas em intervalos. A seguir, o WoE é aplicado para cada um dos intervalos. O mesmo é feito para variaveis categoricas.  <br> 
Para calcular o WoE fazemos: <br>
![](woe.png)

onde:
WoE >0 : a proporcao de bons pagadores é maior que de mal pagadores. <br>
Algumas regras de criacao de intervalos:
1. cada intervalo deve ter ao menos 5% dos dados.  Significa criar de 5-10 intervalos.
2. cada categoria deve ter seu WoE e aquelas que sao similares, devem ser analisadas em conjunto
3. o WOE deve ser monotonico: seja crescente ou então decrescente
4. Valores missing sao analisados separadamente (podemos combinar isso com RF) ou então fazer o preenchimento anteriormente

Antes de seguirmos vamos aprender alguns comandos que serao uteis para construir a formula do WOE:

#### Comando Append
Esse comando é muito util quando usamos um loop para anexar dados a um banco de dados. Imagine o conjunto de dados a seguir com apenas 3 linhas.

In [3]:
dataset=pd.DataFrame({'A':['a',10.0,np.nan],"B":[4.9,5,6]})
dataset

,A,B
0,a,4.9
1,10.0,5.0
2,NaN,6.0


Para anexar uma nova linha, podemos usar o comando "append" da seguinte forma:

In [13]:
dataset=dataset.append({'A':'v','B':7},ignore_index=True)
dataset

,A,B
0,a,4.9
1,10.0,5.0
2,NaN,6.0
3,v,7.0


Agora imagine que queremos anexar mais de uma linha. Nesse caso podemos fazer um loop para anexar (append) mais dados. No exemplo a seguir, construímos uma lista anexando dados dee forma sequencial:

In [4]:
lista=[]
for a in range(0,5):
    lista.append(a)
lista

[0, 1, 2, 3, 4]

Agora podemos fazer isso e criar um banco de dados do tipo pandas

In [5]:
lista2=[]
for b in range(0,5):
    lista2.append({
        'coluna1':b,
        'coluna2':b+1
    })
lista2=pd.DataFrame(lista2)
lista2

,coluna1,coluna2
0,0,1
1,1,2
2,2,3
3,3,4
4,4,5


#### Passos para a construcao do WOE:
A construção dassa estatística fica mais facil de ser entendida se quebrarmos ela em dez passos como mostrado abaixo. Primeiro apresentamos os cinco passos iniciais:

1. selecionar a variavel independente que vai ser analisada
2. descobrir quantas caracteristicas tem nessa variavel
3. quantas vezes aparece a caracteristica (F/M)
4. quantas vezes aparece o resultado (BOM/MAL) para aquela caracteristica
5. criar um dataset

#### Passo 1. Selecionar a variavel independente

In [6]:
df2_=df[['Sex','default']]

#### Passo 2. descobrir quantas caracteristicas tem nessa variavel

In [7]:
carac=df2_['Sex'].nunique()
carac

2

#### Passo 3. Ver a frequencia das características

In [8]:
val=df2_['Sex'].unique()[0]  #caracteristica male
total=df2_[df2_['Sex']==val].shape[0]
total

690

#### Passo 4. Associar esse resultado com a frequencia da variavel dependente

In [10]:
nao=df2_[(df2_['Sex']==val)&(df2_['default']=='no')].shape[0]
sim=total-nao
print("total de nao", nao)
print("total de sim", sim)

total de nao 485
total de sim 205


#### Passo 5. Criar o dataset

In [11]:
lista=[]
lista.append({
    'Carac':val,
    'Total':total,
    'BOM': nao,
    'MAL': sim
})
dataset=pd.DataFrame(lista)
dataset

,Carac,Total,BOM,MAL
0,male,690,485,205


Agora que temos pronta a ideia de como é feito o cálculo da frequencia de cada uma das caracteristicas da variavel, vamos colocar a mesma em uma formula para poder ser aplicada a todas as variaveis. Veja que temos que fazer pequenas modificações na formula como, por exemplo, o loop que ira repetir a formula quantas vezes for necessario para produzir o resultado para todas as caracteristicas da variavel.

In [12]:
def parte1(df,indep,dep,default='no'):
    # df: banco de dados original
    # indep: nome da variavel independente
    # dep: nome da variavel dependente
    # default: caracateristica (nome) dado a pessoa que pagou o credito (no: default)
    s1=time.time()
    df2_=df[[indep,dep]]
    carac=df2_[indep].nunique()
    lista=[]
    for a in range(carac):
        val=df2_[indep].unique()[a]
        total=df2_[df2_[indep]==val].shape[0]
        bom=df2_[(df2_[indep]==val)&(df2_[dep]==default)].shape[0]
        mal=total-bom
        lista.append({
            'Carac':val,
            'Total':total,
            'BOM': bom, #bom pagador
            'MAL': mal})
    dataset=pd.DataFrame(lista)
    s2=time.time()
    print("tempo decorrido: ",s2-s1)
    return dataset

Vamos ver se isso funciona para identificar como seria a distribuição dos dados para a característica sexo entre bom e mal pagador.

In [13]:
sexo=parte1(df,'Sex','default',default='no')
sexo

tempo decorrido:  0.005656003952026367


,Carac,Total,BOM,MAL
0,male,690,485,205
1,female,310,222,88


funcionou!

O que essa formula que criamos faz é simplemente encontrar como que os dados de uma variavel estão distribuidos em suas diferentes caracteristicas e tambem de acordo com a caracteristica da variavel dependente. Vamos aplicar a uma outra variavel para testar:

In [14]:
CA=parte1(df,'Checking account','default',default='no')
CA

tempo decorrido:  0.00655674934387207


,Carac,Total,BOM,MAL
0,little,668,469,199
1,moderate,269,192,77
2,rich,63,46,17


Aqui vale um alerta. Essa função não funciona, por exemplo, para variáveis contínuas como idade e renda. Antes de usar nessas variáveis devemos transformar elas em intervalos (bins). Faremos isso mais a frente. <br>
Após determinar a frequência das características, o proximo passo seria encontrar o WOE para, posteriormente, determinar o IV - Information Value.

6. frequencia do BOM pagador
7. frequencia do MAL pagador
8. aplicar a formula do WOE
9. calcular o IV para cada característica
10. Calcular o IV total de cada variável

partimos do resultado encontrado após aplicar a parte 1 acima:

#### Passo 6. encontrar a frequência com que acontece o evento "nao-default"

In [15]:
sexo['distBom']=sexo['BOM']/(sexo['BOM'].sum())
sexo

,Carac,Total,BOM,MAL,distBom
0,male,690,485,205,0.685997
1,female,310,222,88,0.314003


#### Passo 7. encontrar a frequência do mal pagador "default"

In [16]:
sexo['distMal']=sexo['MAL']/(sexo['MAL'].sum())
sexo

,Carac,Total,BOM,MAL,distBom,distMal
0,male,690,485,205,0.685997,0.699659
1,female,310,222,88,0.314003,0.300341


#### Passo 8. aplicar a fórmula do WOE

In [17]:
sexo['woe']=np.log(sexo['distBom']/sexo['distMal'])
sexo

,Carac,Total,BOM,MAL,distBom,distMal,woe
0,male,690,485,205,0.685997,0.699659,-0.019719
1,female,310,222,88,0.314003,0.300341,0.044483


#### Passo 9. encontrar o IV para cada característica

In [18]:
sexo['IV']=(sexo['distBom']-sexo['distMal'])*sexo['woe']
sexo

,Carac,Total,BOM,MAL,distBom,distMal,woe,IV
0,male,690,485,205,0.685997,0.699659,-0.019719,0.000269
1,female,310,222,88,0.314003,0.300341,0.044483,0.000608


#### Passo 10. encontrando o IV total da variável

In [19]:
IV=sexo['IV'].sum()
print(IV)
sexo

0.0008770929676122427


,Carac,Total,BOM,MAL,distBom,distMal,woe,IV
0,male,690,485,205,0.685997,0.699659,-0.019719,0.000269
1,female,310,222,88,0.314003,0.300341,0.044483,0.000608


Veja que agora o resultado final ficou mais completo, onde temos o percentual de incidência da característica bom pagador e mal pagador para cada uma das características da variável independente analisada. Além disso, temos a estatística woe e iv para essas características. Mas, o que é o IV?

#### IV - Information Value

É um otimo conceito para ser usado na construcao de um credit scorecard e pode dar uma sinalizacao da capacidade preditiva do modelo. O IV é calculado apos termos o resultado do WOE para cada um dos intervalos das variáveis independentes. Assim, de posse do WOE, usamos: <br>
<br>
![](iv.png)
<br>
iv<0.02 --> nao faz sentido usar na previsao  <br>
0.02 - 0.1 --> fraca  <br>
0.1 - 0.3 --> media capacidade de previsao  <br>
maior que 0.3 --> forte previsor   <br>

Agora que temos pronta a lógica da segunda parte, podemos colocar o mesmo em uma função, procurando preservar os mesmos nomes de inputs usados na função da parte 1:

In [20]:
def parte2(df):
    # nome do banco de dados que sai do resultado da parte 1
    s1=time.time()
    df['distBom']=df['BOM']/(df['BOM'].sum())
    df['distMal']=df['MAL']/(df['MAL'].sum())
    df['woe']=np.log(df['distBom']/df['distMal'])
    df['IV']=(df['distBom']-df['distMal'])*df['woe']
    IV=df['IV'].sum()
    print("Information Value",IV)
    s2=time.time()
    print("tempo decorrido: ",s2-s1)
    return df,IV

Vamos testar a nossa função para ver se funciona...

In [21]:
df3_,IV3_=parte2(sexo)
df3_

Information Value 0.0008770929676122427
tempo decorrido:  0.0025568008422851562


,Carac,Total,BOM,MAL,distBom,distMal,woe,IV
0,male,690,485,205,0.685997,0.699659,-0.019719,0.000269
1,female,310,222,88,0.314003,0.300341,0.044483,0.000608


funcionou novamente....

Ao inves de deixar as duas partes em funções distintas, podemos juntar essas duas funções em uma só, facilitando a sua aplicação. Fazemos isso abaixo:

In [22]:
def woe_iv(dataset,indep,dep,default='no'):
    # dataset: banco de dados original
    # indep: nome da variavel independente
    # dep: nome da variavel dependente
    # default: caracateristica (nome) dado a pessoa que pagou o credito
    s1=time.time()
    df2_=dataset[[indep,dep]]
    carac=df2_[indep].nunique()
    lista=[]
    for a in range(carac):
        val=df2_[indep].unique()[a]
        total=df2_[df2_[indep]==val].shape[0]
        bom=df2_[(df2_[indep]==val)&(df2_[dep]==default)].shape[0]
        mal=total-bom
        lista.append({
            'Carac':val,
            'Total':total,
            'BOM': bom,  #bom pagador
            'MAL': mal})
    df=pd.DataFrame(lista)
    #
    df['distBom']=df['BOM']/(df['BOM'].sum())
    df['distMal']=df['MAL']/(df['MAL'].sum())
    df['woe']=np.log(df['distBom']/df['distMal'])
    df['IV']=(df['distBom']-df['distMal'])*df['woe']
    IV=df['IV'].sum()
    print("Information Value " + indep + ':',IV)
    s2=time.time()
    print("tempo decorrido: ",s2-s1)
    return df,IV,indep

e agora vamos testar na variavel sex para ver se o resultado fica igual....

In [23]:
sexo3,iv,indep=woe_iv(dataset=df,indep='Sex',dep='default',default='no')
sexo3

Information Value Sex: 0.0008770929676122427
tempo decorrido:  0.009692192077636719


,Carac,Total,BOM,MAL,distBom,distMal,woe,IV
0,male,690,485,205,0.685997,0.699659,-0.019719,0.000269
1,female,310,222,88,0.314003,0.300341,0.044483,0.000608


Muito bem, agora temos o desafio de criar um loop que ira percorrer o nosso dataset e aplicar a cada uma das variaveis esse calculo e retornar o resultado em um dataframe. Mas antes de fazer isso temos que criar os intervalos (bins) para as nossas variaveis continuas. Para tanto usamos a função qcut() do pandas.

In [26]:
variaveis=['Age','Credit amount','Duration']
df_=df.copy()
for v in variaveis:
    df_[v+'_bin']=pd.qcut(df_[v],5)
    df_.drop([v],axis=1,inplace=True)

In [27]:
df_.head(4)

,Sex,Job,Housing,Saving accounts,Checking account,Purpose,default,Age_bin,Credit amount_bin,Duration_bin
0,male,2,own,little,little,radio/TV,yes,"(45.0, 75.0]","(249.999, 1262.0]","(3.999, 12.0]"
1,female,2,own,little,moderate,radio/TV,no,"(18.999, 26.0]","(4720.0, 18424.0]","(30.0, 72.0]"
2,male,1,own,little,little,education,yes,"(45.0, 75.0]","(1906.8, 2852.4]","(3.999, 12.0]"
3,male,2,free,little,little,furniture/equipment,no,"(36.0, 45.0]","(4720.0, 18424.0]","(30.0, 72.0]"


Agora podemos fazer o loop da nossa função para todo o dataset

In [28]:
woeiv=[]
for c in df_.columns:
    if c=='default': 
        continue
    else:
        woe,iv,var = woe_iv(dataset=df_,indep=c,dep='default',default='no')
        for index, row in woe.iterrows():
            woeiv.append({'variavel':var,
                        'iv_variavel':iv,
                        'Carac':row['Carac'],   
                        'Total':row['Total'],
                        'BOM':row['BOM'],     
                        'MAU':row['MAL'],   
                        'distBom':row['distBom'],  
                        'distMau':row['distMal'],       
                        'woe':row['woe'],        
                        'IV':row['IV'],
                        })
woe2=pd.DataFrame(woeiv)
woe2.tail(6)

Information Value Sex: 0.0008770929676122427
tempo decorrido:  0.009139060974121094
Information Value Job: 0.003090885000194469
tempo decorrido:  0.011207818984985352
Information Value Housing: 0.0034260772034896024
tempo decorrido:  0.012392997741699219
Information Value Saving accounts: 0.03519607892842596
tempo decorrido:  0.01363992691040039
Information Value Checking account: 0.0014675097119170605
tempo decorrido:  0.009927034378051758
Information Value Purpose: 0.014359866945543436
tempo decorrido:  0.01666092872619629
Information Value Age_bin: 0.0041040346737758636
tempo decorrido:  0.019838333129882812
Information Value Credit amount_bin: 0.004990339586237911
tempo decorrido:  0.018032073974609375
Information Value Duration_bin: 0.008030317602742372
tempo decorrido:  0.018218994140625


,variavel,iv_variavel,Carac,Total,BOM,MAU,distBom,distMau,woe,IV
33,Credit amount_bin,0.00499,"(1262.0, 1906.8]",199.0,140.0,59.0,0.198020,0.201365,-0.016753,0.000056
34,Duration_bin,0.00803,"(3.999, 12.0]",359.0,251.0,108.0,0.355021,0.368601,-0.037536,0.000510
35,Duration_bin,0.00803,"(30.0, 72.0]",173.0,122.0,51.0,0.172560,0.174061,-0.008663,0.000013
36,Duration_bin,0.00803,"(15.0, 24.0]",339.0,237.0,102.0,0.335219,0.348123,-0.037771,0.000487
37,Duration_bin,0.00803,"(24.0, 30.0]",57.0,42.0,15.0,0.059406,0.051195,0.148761,0.001222
38,Duration_bin,0.00803,"(12.0, 15.0]",72.0,55.0,17.0,0.077793,0.058020,0.293262,0.005799


O que fizemos acima foi pegar cada variavel do banco de dados, calcular o woe para cada uma das características de uma variável e depois associar o valor de cada linha a respectiva característica. No caso de variáveis contínuas, primeiro criamos um intervalo para depois encontrar o woe e fazer a associação. <br>
Agora temos um banco de dados onde os valores não correspondem mais aos valores originais mas, sim, ao seu respectivo "woe". Sendo assim, reduzimos o escopo de dados ao numero de "woe" que existe em cada variavel. O passo seguinte seria analisar o IV para saber qual dessas variaveis possui um melhor grau de explicação. Mas, antes disso, vamos ver, no notebook 3, como que  tudo o que foi feito acima pode ser facilmente reproduzido a partir de um pacote do python.